# Bayesian putting

You can find more details of this example from
[a case study on the PyMC website](https://www.pymc.io/projects/examples/en/latest/case_studies/putting_workflow.html)

In [ ]:
# Import Python packages
import io

# Import data analysis packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Import scipy
import scipy
import scipy.stats as st

# Import PyMC etc
import arviz as az
import pymc as pm
import xarray as xr

from xarray_einstats.stats import XrContinuousRV

# Other packages
import pytensor.tensor as pt

In [ ]:
# Initialize a random seed for repeatability
RANDOM_SEED = 8927

# Use a "dark grid" style for plotting
az.style.use('arviz-darkgrid')

In [ ]:
# Our hard-coded putting data from Berry (1996)
golf_data = """
distance tries successes
2 1443 1346
3 694 577
4 455 337
5 353 208
6 272 149
7 256 136
8 240 111
9 217 69
10 200 67
11 237 75
12 202 52
13 192 46
14 174 54
15 167 28
16 201 27
17 195 31
18 191 33
19 147 20
20 152 24
"""

golf_data = pd.read_csv(io.StringIO(golf_data), sep=" ", dtype={'distance': 'float'})

BALL_RADIUS = (1.68 / 2) / 12 # radius in feet
CUP_RADIUS = (4.25 / 2) / 12 # cup radius in feet

In [ ]:
# A function to plot our data
def plot_golf_data(golf_data, ax=None, color="C0"):
    """Utility function to standardize a pretty plotting of the golf data."""
    if ax is None:
        _, ax = plt.subplots()
    bg_color = ax.get_facecolor()
    rv = st.beta(golf_data.successes, golf_data.tries - golf_data.successes)
    ax.vlines(golf_data.distance, *rv.interval(0.68), label=None, color=color)
    ax.plot(
        golf_data.distance,
        golf_data.successes / golf_data.tries,
        "o",
        mec=color,
        mfc=bg_color,
        label=None,
    )

    ax.set_xlabel("Distance from hole")
    ax.set_ylabel("Percent of putts made")
    ax.set_ylim(bottom=0, top=1)

    ax.set_xlim(left=0)
    ax.grid(True, axis="y", alpha=0.7)
    return ax

In [ ]:
# And now let's see what it looks like.
ax = plot_golf_data(golf_data)
ax.set_title('Overview of data from Berry (1996)')
plt.show()

## Logit model

Let's fit a traditional logit-binomial model. Our model of successes is

$$
\begin{align*}
    a, b &\sim \mathcal{N(0, 1)} \\
    p(success) &= logit^{-1}(a \cdot distance + b) \\
    num.successes &\sim Binomial(tries, p(success))
\end{align*}
$$

In [ ]:
with pm.Model() as logit_model:
    distance_ = pm.Data("distance", golf_data["distance"], dims="obs_id")
    tries_ = pm.Data("tries", golf_data["tries"], dims="obs_id")
    successes_ = pm.Data("successes", golf_data["successes"], dims="obs_id")

    a_ = pm.Normal("a")
    b_ = pm.Normal("b")

    pm.Binomial(
        "success",
        n=tries_,
        p=pm.math.invlogit(a_ * distance_ + b_),
        observed=successes_,
        dims="obs_id",
    )

In [ ]:
pm.model_to_graphviz(logit_model)

We have some intuition that $a$ should be negative; that is, the longer
the distance, the less likely we are to make a put. We also think that
$b$ should be zero; that is, we expect to make almost 100% of putts of
zero feet. (But this is golf.)

In [ ]:
with logit_model:
    logit_trace = pm.sample()

In [ ]:
az.summary(logit_trace)

In [ ]:
# Draw posterior predictive samples
with logit_model:
    logit_trace.extend(pm.sample_posterior_predictive(logit_trace))

# Hard to plot more than 400 sensibly
logit_post = az.extract(logit_trace, num_samples=400)
logit_ppc = az.extract(logit_trace, group="posterior_predictive", num_samples=400)
const_data = logit_trace["constant_data"]

logit_ppc_success = logit_ppc["success"] / const_data["tries"]

# Plotting
ax = plot_golf_data(golf_data)
t_ary = np.linspace(CUP_RADIUS - BALL_RADIUS, golf_data.distance.max(), 200)
t = xr.DataArray(t_ary, coords=[("distance", t_ary)])
logit_post["expit"] = scipy.special.expit(logit_post["a"] * t + logit_post["b"])

ax.plot(
    t,
    logit_post["expit"].T,
    lw=1,
    color="C1",
    alpha=0.5,
)

ax.plot(t, logit_post["expit"].mean(dim="sample"), color="C2")

ax.plot(golf_data.distance, logit_ppc_success, "k.", alpha=0.01)
ax.set_title("Logit mean and posterior predictive")
plt.show()

The fit is okay, but it is not especially good. This observation
is not unexpected. In Bayesian analysis, we start with **simple**
models and "add complexity" as needed.

## Geometry-based model

We will assume that professional golfers can hit the ball in a
specific direction with some (small?) error. In other words,
the angle the ball actually travels is normally distributed
around 0 with some variance we will try to learn.

The ball goes in whenever the error in the angle is small
enough that the center of gravity of the ball is inside the
cup.

Here is the probability of sinking a put:

$$
p(success | \sigma_{angle}, distance) = 2 \Phi \left(\frac {arcsin((R - r)/distance} {\sigma_{angle}} \right)
$$

where $Phi$ is the normal cumulative density function, R is the radius of the cup and r is the radius of the ball.

In [ ]:
def phi(x):
    """Calculates the standard normal cumulative distribution function."""
    return 0.5 + 0.5 * pt.erf(x / pt.sqrt(2.0))


with pm.Model() as angle_model:
    distance_ = pm.MutableData("distance", golf_data["distance"], dims="obs_id")
    tries_ = pm.MutableData("tries", golf_data["tries"], dims="obs_id")
    successes_ = pm.MutableData("successes", golf_data["successes"], dims="obs_id")

    variance_of_shot = pm.HalfNormal("variance_of_shot")
    p_goes_in = pm.Deterministic(
        "p_goes_in",
        2 * phi(pt.arcsin((CUP_RADIUS - BALL_RADIUS) / distance_) / variance_of_shot) - 1,
        dims="obs_id",
    )
    success = pm.Binomial("success", n=tries_, p=p_goes_in, observed=successes_, dims="obs_id")

In [ ]:
pm.model_to_graphviz(angle_model)

### Prior Predictive Checks

In [ ]:
with angle_model:
    angle_trace = pm.sample_prior_predictive(500)

In [ ]:
with angle_model:
    angle_trace.extend(pm.sample(1000, tune=1000, target_accept=0.85))

angle_post = az.extract(angle_trace)

In [ ]:
def forward_angle_model(variances_of_shot, t):
    norm_dist = XrContinuousRV(st.norm, 0, variances_of_shot)
    return 2 * norm_dist.cdf(np.arcsin((CUP_RADIUS - BALL_RADIUS) / t)) - 1

In [ ]:
ax = plot_golf_data(golf_data)

angle_post["expit"] = forward_angle_model(angle_post["variance_of_shot"], t)

ax.plot(
    t,
    angle_post["expit"][:, ::100],
    lw=1,
    color="C1",
    alpha=0.1,
)

ax.plot(
    t,
    angle_post["expit"].mean(dim="sample"),
    label="Geometry-based model",
)

ax.plot(
    t,
    logit_post["expit"].mean(dim="sample"),
    label="Logit-binomial model",
)
ax.set_title("Comparing the fit of geometry-based and logit-binomial model")
ax.legend()
plt.show()

The new model appears to fit much better. This model suggests that
a 50 foot putt has a much higher chance of going in.